# Creating a Customer Service Agent with Client-Side Tools

In this recipe, we'll demonstrate how to create a customer service chatbot using Claude plus client-side tools. The chatbot will be able to look up customer information, retrieve order details, and cancel orders on behalf of the customer. We'll use the SDK's tool runner to automatically handle the agentic loop.

## Step 1: Set up the environment

First, let's install the required libraries and set up the Claude API client.

In [ ]:
%pip install anthropic

In [ ]:
from anthropic import Anthropic, beta_tool

client = Anthropic()
MODEL_NAME = "claude-sonnet-4-5-20250929"

## Step 2: Define the client-side tools

Next, we'll define the client-side tools that our chatbot will use to assist customers. We'll create three tools using the `@beta_tool` decorator: `get_customer_info`, `get_order_details`, and `cancel_order`.

In [ ]:
import json

# Simulated customer data
customers = {
    "C1": {"name": "John Doe", "email": "john@example.com", "phone": "123-456-7890"},
    "C2": {"name": "Jane Smith", "email": "jane@example.com", "phone": "987-654-3210"},
}

# Simulated order data
orders = {
    "O1": {
        "id": "O1",
        "product": "Widget A",
        "quantity": 2,
        "price": 19.99,
        "status": "Shipped",
    },
    "O2": {
        "id": "O2",
        "product": "Gadget B",
        "quantity": 1,
        "price": 49.99,
        "status": "Processing",
    },
}


@beta_tool
def get_customer_info(customer_id: str) -> str:
    """Retrieves customer information based on their customer ID.

    Args:
        customer_id: The unique identifier for the customer.

    Returns:
        Customer's name, email, and phone number as JSON.
    """
    customer = customers.get(customer_id, None)
    if customer:
        return json.dumps(customer)
    return json.dumps({"error": "Customer not found"})


@beta_tool
def get_order_details(order_id: str) -> str:
    """Retrieves the details of a specific order based on the order ID.

    Args:
        order_id: The unique identifier for the order.

    Returns:
        Order ID, product name, quantity, price, and order status as JSON.
    """
    order = orders.get(order_id, None)
    if order:
        return json.dumps(order)
    return json.dumps({"error": "Order not found"})


@beta_tool
def cancel_order(order_id: str) -> str:
    """Cancels an order based on the provided order ID.

    Args:
        order_id: The unique identifier for the order to be cancelled.

    Returns:
        Confirmation message if the cancellation is successful.
    """
    if order_id in orders:
        return json.dumps({"success": True, "message": f"Order {order_id} has been cancelled"})
    return json.dumps({"success": False, "error": "Order not found"})


# List of tools for the tool runner
tools = [get_customer_info, get_order_details, cancel_order]

## Step 3: Interact with the chatbot using the Tool Runner

Now, let's create a function to interact with the chatbot using the SDK's tool runner. The tool runner automatically handles the agentic loop - executing tools when Claude requests them and continuing the conversation until Claude provides a final response.

In [ ]:
def chatbot_interaction(user_message):
    print(f"\n{'=' * 50}\nUser Message: {user_message}\n{'=' * 50}")

    # Use the tool runner to handle the agentic loop automatically
    runner = client.beta.messages.tool_runner(
        model=MODEL_NAME,
        max_tokens=4096,
        tools=tools,
        messages=[{"role": "user", "content": user_message}],
    )

    # Iterate through responses - the runner executes tools automatically
    for message in runner:
        print(f"\nResponse (stop_reason: {message.stop_reason}):")
        for block in message.content:
            if hasattr(block, "text"):
                print(f"Text: {block.text}")
            elif block.type == "tool_use":
                print(f"Tool Used: {block.name}")
                print(f"Tool Input: {block.input}")

    # The final message contains Claude's response after all tool use
    final_response = next(
        (block.text for block in message.content if hasattr(block, "text")),
        None,
    )
    print(f"\nFinal Response: {final_response}")
    return final_response

## Step 4: Test the chatbot
Let's test our customer service chatbot with a few sample queries.

In [17]:
chatbot_interaction("Can you tell me the email address for customer C1?")
chatbot_interaction("What is the status of order O2?")
chatbot_interaction("Please cancel order O1 for me.")


User Message: Can you tell me the email address for customer C1?

Initial Response:
Stop Reason: tool_use
Content: [ContentBlock(text='<thinking>The get_customer_info function retrieves a customer\'s name, email, and phone number given their customer ID. To call this function, I need the customer_id parameter. The user provided the customer ID "C1" in their request, so I have the necessary information to make the API call.</thinking>', type='text'), ContentBlockToolUse(id='toolu_019F9JHokMkJ1dHw5BEh28sA', input={'customer_id': 'C1'}, name='get_customer_info', type='tool_use')]

Tool Used: get_customer_info
Tool Input:
{
  "customer_id": "C1"
}

Tool Result:
{
  "name": "John Doe",
  "email": "john@example.com",
  "phone": "123-456-7890"
}

Response:
Stop Reason: end_turn
Content: [ContentBlock(text='The email address for customer C1 (John Doe) is john@example.com.', type='text')]

Final Response: The email address for customer C1 (John Doe) is john@example.com.

User Message: What is 

'Based on the confirmation received, your order O1 has been successfully cancelled. Please let me know if there is anything else I can assist you with.'

And that's it! We've created a customer service chatbot using Claude and client-side tools with the SDK's tool runner. The tool runner automatically handles the agentic loop - executing tools when Claude requests them and continuing the conversation until Claude provides a final response.

The chatbot can look up customer information, retrieve order details, and cancel orders based on the user's requests. By using the `@beta_tool` decorator, we define tools with clear descriptions extracted from docstrings, enabling Claude to effectively understand and utilize the available tools.

Feel free to expand on this example by integrating with your actual customer database and order management system, and by adding more tools to handle a wider range of customer service tasks.